amocrm api was of version 2.4.1 when statuses download was not working

In [1]:
! pip freeze | grep amo

amocrm-api==2.5.0
amocrm-api-wrapper==0.0.16


In [1]:
! pip install -U amocrm_api

  Using cached amocrm_api-2.5.0-py3-none-any.whl (32 kB)
  Attempting uninstall: amocrm_api
    Found existing installation: amocrm-api 2.4.1
    Uninstalling amocrm-api-2.4.1:
      Successfully uninstalled amocrm-api-2.4.1


# LOGON

In [1]:
from amocrm.v2 import tokens
from amocrm.v2.exceptions import UnAuthorizedException, AmoApiException

In [2]:
from franchize_config import keys
keys

['d244240e-a192-48ef-a5ce-e6744b0a3b17',
 'bRLRcyWeSVJMsp6Ny8Mu8cKvkoRF80qlAUKXnP33PzaLuW0EDUzSqRgEUbZHqizX',
 'yastaff',
 'https://yastaff.amocrm.ru/']

In [3]:
from franchize_config import keys
tokens.default_token_manager(
    *keys,
    storage=tokens.FileTokensStorage(directory_path="./test_dir/franchize"),  # by default FileTokensStorage
)

Don't touch it, until you need to initialize new keys generation

In [5]:
tokens.default_token_manager.init(
    code='',
    skip_error=False
)

# Customs fields

ok, custom fields are working perfectly fine!§

In [4]:
from amocrm.v2 import filters
from models import Lead, Company, Contact
import time, datetime

In [5]:
lead = Lead.objects.get(object_id="44711174")


In [6]:
lead.tip_traffika.__dict__["value"]
lead.telefon

'+7 (913) 892-52-63'

In [5]:
from requests.adapters import HTTPAdapter
from amocrm.v2.interaction import _session
_session.mount("https://", HTTPAdapter(max_retries=5))

Мы тут в дефолтную сессию requests которая используется для запросов стандартными механизмами requests добавляем ретраи.

In [6]:
leads = list(Lead.objects.all())

In [7]:
len(leads)

144531

Connection sometimes gets aborted, let's see about all leads, but for daily load we need to work on some exceptions;)

# Parsing leads

In [8]:
import pandas as pd

In [43]:
lead.rassrochka.__di

In [9]:
tags_list = lead.tags.__dict__["_tags"]
tag_str = ""
for tag in tags_list:
    if tag_str != "":
        tag_str += f', {tag["name"]}'
    else:
        tag_str += tag["name"]

NameError: name 'lead' is not defined

In [35]:
tag_str

'Лид целевой, Закрыт на блок бабло, Получил звонок от бота, ЛР ЛЧ'

In [10]:
leads_dicts = []
counter = 0
for lead in leads:
    leads_dicts.append(lead.__dict__['_data'])
    try:
        leads_dicts[counter]["traffic_type"] = lead.tip_traffika.__dict__["value"]
    except AttributeError:
        leads_dicts[counter]["traffic_type"] = ""
    leads_dicts[counter]["city"] = lead.gorod
    try:
        leads_dicts[counter]["resource"] = lead.resurs.__dict__["value"]
    except AttributeError:
        leads_dicts[counter]["resource"] = ""
    try:
        leads_dicts[counter]["loss_reason"] = lead.prichina_otkola.__dict__["value"]
    except AttributeError:
        leads_dicts[counter]["loss_reason"] = ""
    try:
        leads_dicts[counter]["package"] = lead.paket.__dict__["value"]
    except AttributeError:
        leads_dicts[counter]["package"] = ""
    try:
        leads_dicts[counter]["gender"] = lead.pol.__dict__["value"]
    except AttributeError:
        leads_dicts[counter]["gender"] = ""
    try:
        leads_dicts[counter]["kev_ot"] = lead.kev_ot
    except AttributeError:
        leads_dicts[counter]["kev_ot"] = ""
    try:
        leads_dicts[counter]["type"] = lead.type.__dict__["value"]
    except AttributeError:
        leads_dicts[counter]["type"] = ""
    try:
        leads_dicts[counter]["agreement_no"] = lead.nomer_dogovora
    except AttributeError:
        leads_dicts[counter]["agreement_no"] = ""
    try:
        leads_dicts[counter]["installment_plan"] = lead.rassrochka.__dict__["value"]
    except AttributeError:
        leads_dicts[counter]["installment_plan"] = ""
    try:
        leads_dicts[counter]["presale"] = str(lead.presale)
    except AttributeError:
        leads_dicts[counter]["isntallment_plan"] = ""
    tags_list = lead.tags.__dict__["_tags"]
    tag_str = ""
    for tag in tags_list:
        if tag_str != "":
            tag_str += f', {tag["name"]}'
        else:
            tag_str += tag["name"]
    leads_dicts[counter]["tags"] = tag_str
    counter += 1

In [17]:
leads_dicts[1]

{'id': 32945010,
 'name': 'Максим Москва',
 'price': 299000,
 'responsible_user_id': 828267,
 'group_id': 132579,
 'status_id': 142,
 'pipeline_id': 15219,
 'loss_reason_id': None,
 'created_by': 326314,
 'updated_by': 363811,
 'created_at': 1458636317,
 'updated_at': 1507122056,
 'closed_at': 1461319317,
 'closest_task_at': None,
 'is_deleted': False,
 'custom_fields_values': [{'field_id': 1279825,
   'field_name': 'Ресурс',
   'field_code': None,
   'field_type': 'select',
   'values': [{'value': 'fr.yastaff', 'enum_id': 2943049, 'enum_code': None}]},
  {'field_id': 1279829,
   'field_name': 'Мусор (не трогаем)',
   'field_code': None,
   'field_type': 'select',
   'values': [{'value': 'Заявка', 'enum_id': 2943061, 'enum_code': None}]},
  {'field_id': 1280257,
   'field_name': 'Занятость',
   'field_code': None,
   'field_type': 'select',
   'values': [{'value': '100%', 'enum_id': 2943965, 'enum_code': None}]},
  {'field_id': 1275421,
   'field_name': 'Дата прихода обращения',
   'fi

In [18]:
leads_dicts[0]["custom_fields_values"] == None

False

In [57]:
thirty_eight = []
for i in leads_dicts:
    if i["custom_fields_values"] != None:
        if (len(i["custom_fields_values"])) == 38:
            thirty_eight.append(i)

In [48]:
thirty_eight[0]["custom_fields_values"][0]["values"][0]["value"]

'fr.yastaff/plan'

In [11]:
custom_fields_needed = [
    "Дата прихода обращения", "utm_source", "utm_medium", "utm_campaign", 
    "utm_term", "utm_content", "utm_hueisment", "utm_campaign (2)", 
    "utm_term (2)"
]

In [12]:
for lead in leads_dicts:
    if lead["custom_fields_values"] != None:
        for field in lead["custom_fields_values"]:
            for field_name in custom_fields_needed:
                if field["field_name"] == field_name:
                    lead[field_name] = field["values"][0]["value"]

In [13]:
df_leads = pd.DataFrame(leads_dicts)

In [14]:
df_leads["data_obrasheniya"] = df_leads["Дата прихода обращения"].fillna(0).astype("int")
df_leads.drop([
    "_links", "_embedded", "custom_fields_values", "Дата прихода обращения"
], axis=1, inplace=True)
df_leads.rename({"price": "budget"}, axis=1)
df_leads["created_at"] = pd.to_datetime(df_leads["created_at"], unit="s")
df_leads["updated_at"] = pd.to_datetime(df_leads["updated_at"], unit="s")
df_leads["closed_at"] = pd.to_datetime(df_leads["closed_at"], unit="s")


In [62]:
df_leads[["created_at"]]

,created_at
0,2016-02-25 13:57:41
1,2016-03-22 08:45:17
2,2016-03-22 09:40:45
3,2016-03-28 13:41:55
4,2016-03-31 06:24:11
...,...
144476,2022-10-07 09:26:04
144477,2022-10-12 10:09:22
144478,2022-10-14 08:38:23
144479,2022-10-17 11:57:33


In [15]:
df_leads.rename({
    "utm_campaign (2)": "utm_campaign_2",
    "utm_term (2)": "utm_term_2"
}, axis=1, inplace=True)

In [18]:
df_leads["installment_plan"].unique()

array(['', 'Нет', 'Да'], dtype=object)

In [19]:
df_leads.dtypes

id                              int64
name                           object
price                           int64
responsible_user_id             int64
group_id                        int64
status_id                       int64
pipeline_id                     int64
loss_reason_id                 object
created_by                      int64
updated_by                      int64
created_at             datetime64[ns]
updated_at             datetime64[ns]
closed_at              datetime64[ns]
closest_task_at               float64
is_deleted                       bool
score                          object
account_id                      int64
labor_cost                     object
traffic_type                   object
city                           object
resource                       object
loss_reason                    object
package                        object
gender                         object
kev_ot                         object
type                           object
agreement_no

In [20]:
df_leads.columns

Index(['id', 'name', 'price', 'responsible_user_id', 'group_id', 'status_id',
       'pipeline_id', 'loss_reason_id', 'created_by', 'updated_by',
       'created_at', 'updated_at', 'closed_at', 'closest_task_at',
       'is_deleted', 'score', 'account_id', 'labor_cost', 'traffic_type',
       'city', 'resource', 'loss_reason', 'package', 'gender', 'kev_ot',
       'type', 'agreement_no', 'installment_plan', 'presale', 'tags',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_term', 'utm_content',
       'utm_campaign_2', 'utm_term_2', 'data_obrasheniya'],
      dtype='object')

# BigQuery

In [3]:
pip install google-cloud-bigquery

  Using cached google_cloud_bigquery-3.3.2-py2.py3-none-any.whl (211 kB)
  Using cached pyarrow-9.0.0-cp310-cp310-macosx_10_13_x86_64.whl (24.0 MB)
  Using cached google_api_core-2.10.0-py3-none-any.whl (115 kB)
  Using cached proto_plus-1.22.1-py3-none-any.whl (47 kB)
  Using cached google_resumable_media-2.3.3-py2.py3-none-any.whl (76 kB)
  Using cached google_cloud_bigquery_storage-2.15.0-py2.py3-none-any.whl (182 kB)
  Using cached protobuf-4.21.5-cp37-abi3-macosx_10_9_universal2.whl (484 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 5.0 MB/s eta 0:00:0000:0100:01
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Using cached googleapis_common_protos-1.56.4-py2.py3-none-any.whl (211 kB)
  Using cached grpcio_status-1.48.1-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
! pip install pandas-gbq

In [21]:
df_leads.to_csv("leads.csv", index=False)

In [23]:
import pandas as pd
df_leads = pd.read_csv("leads.csv", low_memory=False)

In [24]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./oddjob-db-2007-759fe782b144.json"

In [68]:
from google.cloud import bigquery


In [69]:
client = bigquery.Client()

In [70]:
df_leads.dtypes

id                       int64
name                    object
price                    int64
responsible_user_id      int64
group_id                 int64
status_id                int64
pipeline_id              int64
loss_reason_id         float64
created_by               int64
updated_by               int64
created_at              object
updated_at              object
closed_at               object
closest_task_at        float64
is_deleted                bool
score                  float64
account_id               int64
labor_cost             float64
traffic_type            object
city                    object
resource                object
loss_reason             object
package                 object
gender                  object
kev_ot                  object
type                   float64
agreement_no            object
isntallment_plan       float64
presale                 object
tags                    object
utm_source              object
utm_medium              object
utm_camp

In [5]:
    schema = [
        {'name': 'id', 'type': 'INTEGER', 'mode': 'REQUIRED'},
        {'name': 'name', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'price', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'responsible_user_id', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'group_id', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'status_id', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'pipeline_id', 'type': 'INTEGER', 'mode': 'REQUIRED'},
        {'name': 'loss_reason_id', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'created_by', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'updated_by', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'created_at', 'type': 'TIMESTAMP', 'mode': 'REQUIRED'},
        {'name': 'updated_at', 'type': 'TIMESTAMP', 'mode': 'REQUIRED'},
        {'name': 'closet_at', 'type': 'TIMESTAMP', 'mode': 'NULLABLE'},
        {'name': 'closest_task_at', 'type': 'FLOAT', 'mode': 'NULLABLE'},
        {'name': 'is_deleted', 'type': 'BOOLEAN', 'mode': 'NULLABLE'},
        {'name': 'score', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'account_id', 'type': 'INTEGER', 'mode': 'NULLABLE'},
        {'name': 'labor_cost', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'traffic_type', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'city', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'resource', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'package', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'utm_source', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'utm_medium', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'utm_campaign', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'utm_term', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'utm_content', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'utm_hueisment', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'utm_campaign_2', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'utm_term_2', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'data_obrasheniya', 'type': 'INTEGER', 'mode': 'NULLABLE'}
    ]

id                              int64    id                              int64
name                           object    name                           object
budget                          int64    price                           int64
responsible_user_id             int64    responsible_user_id             int64
group_id                        int64    group_id                        int64
status_id                       int64    status_id                       int64
pipeline_id                     int64    pipeline_id                     int64
loss_reason_id                float64    loss_reason_id                float64
created_by                      int64    created_by                      int64
updated_by                      int64    updated_by                      int64
created_at             datetime64[ns]    created_at             datetime64[ns]
updated_at             datetime64[ns]    updated_at             datetime64[ns]
closed_at              datetime64[ns]    closed_at              datetime64[ns]
closest_task_at               float64    closest_task_at               float64
is_deleted                       bool    is_deleted                       bool
score                         float64    score                         float64
account_id                      int64    account_id                      int64
labor_cost                    float64    labor_cost                    float64
traffic_type                   object    traffic_type                   object
city                           object    city                           object
resource                       object    resource                       object
package                        object    package                        object
utm_source                     object    utm_source                     object
utm_medium                     object    utm_medium                     object
utm_campaign                   object    utm_campaign                   object
utm_term                       object    utm_term                       object
utm_content                    object    utm_content                    object
utm_hueisment                  object    utm_hueisment                  object
utm_campaign_2                 object    utm_campaign_2                 object
utm_term_2                     object    utm_term_2                     object
data_obrasheniya                int64    data_obrasheniya                int64

In [9]:
pip install -U pandas-gbq

Note: you may need to restart the kernel to use updated packages.


In [25]:
# df_leads = df_leads.astype("str")
df_leads.to_gbq( "franchise_oddjob.dw_amocrm_fr_leads", if_exists="replace",
                    # table_schema=schema
                    )

In [68]:
start = time.time()
end = time.time()

In [27]:
end - start

10.618908882141113

# Gspread events

In [4]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./oddjob-db-2007-759fe782b144.json"

In [5]:
! echo $GOOGLE_APPLICATION_CREDENTIALS

./oddjob-db-2007-759fe782b144.json


In [5]:
from google.cloud import bigquery

In [6]:
client = bigquery.Client()

In [1]:
import pandas as pd

In [2]:
df_events = pd.read_csv("statuses_changes.csv")

In [7]:
df_events.to_gbq(
    "franchise_oddjob.dw_amocrm_fr_events", if_exists="replace"
    )

# Filters

In [5]:
from amocrm.v2 import filters
from models import Lead, Company, Contact
from datetime import timedelta, date
import datetime

In [6]:
now = datetime.datetime.now()

In [7]:
leads = list(Lead.objects.filter(filters=[filters.DateRangeFilter("updated_at")(now-timedelta(days=2), now)]))

# Events

In [4]:
from amocrm.v2 import filters
from amocrm.v2 import Event
from datetime import timedelta, date
import datetime, time

In [4]:
now = datetime.datetime.now()

In [5]:
from requests.adapters import HTTPAdapter
from amocrm.v2.interaction import _session
_session.mount("https://", HTTPAdapter(max_retries=5))

### TODO updated_at vs created_at

In [28]:
datetime.datetime.today()

datetime.datetime(2022, 7, 24, 0, 7, 57, 584719)

In [16]:
datetime.datetime(2022, 7, 23, 0, 0) - timedelta(minutes=90)

datetime.datetime(2022, 7, 22, 22, 30)

In [11]:
start = time.time()
events = list(Event.objects.filter(
    filters=[
        filters.DateRangeFilter("created_at")(now-timedelta(days=7), now),
        filters.SingleFilter("type")("lead_status_changed")
    ]
))
end = time.time()

In [8]:
events = list(Event.objects.all())

In [5]:
(end - start) / 60

0.010429100195566813

In [12]:
len(events)

50

# Parsing Events

In [40]:
import pandas as pd

In [26]:
events_dicts = []
for event in events:
    events_dicts.append(event.__dict__['_data'])

In [27]:
events_dicts[0].keys()

dict_keys(['id', 'type', 'entity_id', 'entity_type', 'created_by', 'created_at', 'value_after', 'value_before', 'account_id', '_links', '_embedded'])

In [28]:
events_dicts[0]["value_after"][0]["lead_status"]

{'id': 41469585, 'pipeline_id': 4483554}

In [29]:
for event in events_dicts:
    event["id_status_before"] = event["value_before"][0]["lead_status"]["id"]
    event["id_pipeline_before"] = event["value_before"][0]["lead_status"]["pipeline_id"]
    event["id_status_after"] = event["value_after"][0]["lead_status"]["id"]
    event["id_pipeline_after"] = event["value_after"][0]["lead_status"]["pipeline_id"]

In [30]:
df_events = pd.DataFrame(events_dicts)

In [31]:
df_events.drop([
    "value_after", "value_before", "_links", "_embedded"
], axis=1, inplace=True)
df_events["created_at"] = pd.to_datetime(df_events["created_at"], unit="s")

In [38]:
df_events.to_csv("evetns.csv", index=False)

# Events to BQ

In [4]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./oddjob-db-2007-759fe782b144.json"

In [5]:
! echo $GOOGLE_APPLICATION_CREDENTIALS

./oddjob-db-2007-759fe782b144.json


In [6]:
from google.cloud import bigquery

In [7]:
client = bigquery.Client()

In [34]:
df_events.to_gbq(
    "franchise_oddjob.dw_amocrm_fr_events", if_exists="append"
)

# Getting pipelines

In [3]:
from amocrm.v2 import Pipeline

In [6]:
pipelines = list(Pipeline.objects.all())

In [9]:
pipelines_dicts = []
for pipeline in pipelines:
    pipelines_dicts.append(pipeline.__dict__['_data'])

In [7]:
import pandas as pd

In [10]:
df_pipelines = pd.DataFrame(pipelines_dicts)

### TODO statuses to separate table

In [11]:
df_pipelines.drop(["_links", "_embedded"], axis=1, inplace=True)

In [12]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./oddjob-db-2007-759fe782b144.json"

In [13]:
from google.cloud import bigquery
client = bigquery.Client()

In [15]:
df_pipelines.to_gbq(
    "franchise_oddjob.dw_amocrm_fr_pipelines",
    if_exists="replace"
)

# Calls

In [25]:
from amocrm.v2 import Call, CallStatus, CallDuration

In [37]:
Call.get_all('test')

TypeError: 'NotImplementedType' object is not callable

# Users

In [3]:
from amocrm.v2 import User

In [4]:
users = list(User.objects.all())

In [5]:
users

[User(catalog_access = True, email = nikolay.k@gk-pr.ru, id = 600075, is_active = True, is_admin = True, is_free = False, lang = ru, mail_access = True, name = Персональное решение, status_rights = [{'entity_type': 'leads', 'pipeline_id': 15219, 'status_id': 23406784, 'rights': {'view': 'A', 'edit': 'A', 'delete': 'A'}}, {'entity_type': 'leads', 'pipeline_id': 1041060, 'status_id': 23406781, 'rights': {'view': 'A', 'edit': 'A', 'delete': 'A'}}, {'entity_type': 'leads', 'pipeline_id': 1045806, 'status_id': 23406787, 'rights': {'view': 'A', 'edit': 'A', 'delete': 'A'}}, {'entity_type': 'leads', 'pipeline_id': 1443795, 'status_id': 23406790, 'rights': {'view': 'A', 'edit': 'A', 'delete': 'A'}}, {'entity_type': 'leads', 'pipeline_id': 1520397, 'status_id': 23406793, 'rights': {'view': 'A', 'edit': 'A', 'delete': 'A'}}, {'entity_type': 'leads', 'pipeline_id': 1981812, 'status_id': 29378454, 'rights': {'view': 'A', 'edit': 'A', 'delete': 'A'}}, {'entity_type': 'leads', 'pipeline_id': 2180478

## Parsing Users

In [6]:
import pandas as pd

In [14]:
users_dicts = []
for user in users:
    users_dicts.append(user.__dict__['_data'])

In [22]:
users_dicts[0]['rights'].keys()

dict_keys(['leads', 'contacts', 'companies', 'tasks', 'mail_access', 'catalog_access', 'files_access', 'status_rights', 'catalog_rights', 'custom_fields_rights', 'oper_day_reports_view_access', 'oper_day_user_tracking', 'is_admin', 'is_free', 'is_active', 'group_id', 'role_id'])

In [25]:
for user in users_dicts:
    user['is_admin'] = users_dicts[0]['rights']['is_admin']
    user['is_active'] = users_dicts[0]['rights']['is_active']
    user['group_id'] = users_dicts[0]['rights']['group_id']
    user['role_id'] = users_dicts[0]['rights']['role_id']

In [24]:
users_dicts[0]['rights']['is_active']

True

In [26]:
df_users = pd.DataFrame(users_dicts)

In [27]:
df_users.drop(["rights", "_links"], axis=1, inplace=True)

In [29]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./oddjob-db-2007-759fe782b144.json"

In [30]:
! echo $GOOGLE_APPLICATION_CREDENTIALS

./oddjob-db-2007-759fe782b144.json


In [31]:
from google.cloud import bigquery

In [32]:
client = bigquery.Client()

In [33]:
df_users.to_gbq(
    "franchise_oddjob.dw_amocrm_fr_users", if_exists="replace"
)

# Status names

In [50]:
from amocrm.v2 import Status, filters
from amocrm.v2.exceptions import AmoApiException

In [7]:
import pandas as pd

In [ ]:
df_statuses = pd.DataFrame.from_dict({
    'id': ['']
})

In [22]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./oddjob-db-2007-759fe782b144.json"

In [23]:
! echo $GOOGLE_APPLICATION_CREDENTIALS

./oddjob-db-2007-759fe782b144.json


In [24]:
from google.cloud import bigquery

In [51]:
df_temp_pipe = pd.read_gbq(
    """
    SELECT DISTINCT
        id
    FROM
        `oddjob-db-2007.franchise_oddjob.dw_amocrm_fr_pipelines`
    """
)
pipeline_ids = df_temp_pipe['id'].to_list()
statuses = []
for pipeline in pipeline_ids:
    df_temp_stat = pd.read_gbq(
        f"""
        SELECT DISTINCT
            id_status_before
        FROM
            `oddjob-db-2007.franchise_oddjob.dw_amocrm_fr_events`
        WHERE id_pipeline_before = {pipeline}
        """
    )
    status_ids = df_temp_stat["id_status_before"].to_list()
    for status in status_ids:
        try:
            statuses.append(
                Status.get_for(f"{pipeline}").get(f"{status}").__dict__["_data"]
            )
        except AmoApiException:
            pass

In [52]:
df_statuses = pd.DataFrame(statuses)

In [54]:
df_statuses.drop(["_links"], axis=1, inplace=True)

In [56]:
df_statuses.to_gbq(
    'franchise_oddjob.dw_amocrm_fr_statuses',
    if_exists='replace'
)